In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from Regressor import RegressorWrapper
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import logging
import re
import sqlite3
#Définition de constantes
AXE_COLONNES = 1

## Sources

| Sources_données_originales | Date_de_mise_à_jour |
| ---------------------------|---------------------|
| https://www.insee.fr/fr/statistiques/serie/010760415#Telechargement | 26/01/23 |
| https://www.insee.fr/fr/statistiques/serie/010760376#Telechargement | 26/01/23 |
| https://www.insee.fr/fr/statistiques/7076724#onglet-1 | 11/04/23 |
| https://www.insee.fr/fr/statistiques/7076722 | 07/04/23 |
| https://www.insee.fr/fr/statistiques/serie/010760261#Telechargement | 26/01/23 |
| https://www.insee.fr/fr/statistiques/serie/010760364#Telechargement | 26/01/23 |
| https://www.insee.fr/fr/statistiques/serie/010759840#Telechargement | 26/01/23 |
| https://www.insee.fr/fr/statistiques/serie/010759801#Telechargement | 26/01/23 |
| https://www.insee.fr/fr/statistiques/serie/010760261 | 26/01/23 |
| https://www.insee.fr/fr/statistiques/serie/010759955 | 26/01/23 |
| https://www.insee.fr/fr/statistiques/serie/010760300 | 26/01/23 |
| https://www.insee.fr/fr/statistiques/serie/010759801 | 26/01/23 |
| https://www.data.gouv.fr/fr/datasets/election-presidentielle-des-10-et-24-avril-2022-resultats-definitifs-du-1er-tour/ | 14/04/2023 |
| https://www.data.gouv.fr/fr/datasets/election-presidentielle-des-23-avril-et-7-mai-2017-resultats-du-1er-tour/ | 24/04/2017 |
| https://www.data.gouv.fr/fr/datasets/election-presidentielle-2012-resultats-572124/ | 06/01/2014 |
| https://www.data.gouv.fr/fr/datasets/election-presidentielle-2007-resultats-572120/ | 06/01/2014 |
| https://www.data.gouv.fr/fr/datasets/bases-statistiques-communale-et-departementale-de-la-delinquance-enregistree-par-la-police-et-la-gendarmerie-nationales/ | 09/03/2023 |

# Traitement des données sur l'emploi
## Obtention des données

In [ ]:
sns.set(rc={'figure.figsize':(20,8.27)})

In [ ]:
def separer_numero_departement(valeur):
    valeur_a_comparer = valeur.strip().lower()
    if valeur_a_comparer == "loire atlantique":
       return 44
    elif valeur_a_comparer == "alpes-maritimes":
       return 6

In [ ]:
def afficher_noms_colonnes_avec_valeurs_manquantes(dataframe):
  column_labels = list(dataframe.columns)
  for column_label in column_labels:
    if dataframe[column_label].hasnans:
      print(column_label)

In [ ]:
sheet_names = ['Emploi LA', 'Emploi Indus LA', 'Tertiaire marchand LA', 'Tertiaire non march LA', 'Chom LA', 'Emploi AM', 'Emploi Indus AM', 'Tertiaire march AM', 'Tertiaire non march AM OK', 'Chom AM']
dict_donnees_insee = pd.read_excel("./data/donnees_insee.xlsx", sheet_name=sheet_names, skiprows=6)
dict_donnees_insee['Tertiaire non march AM OK'] = dict_donnees_insee['Tertiaire non march AM OK'].drop(33, axis=0)

for item in dict_donnees_insee.items():
   dataframe = dict_donnees_insee[item[0]]
   dataframe = dataframe.dropna(axis=0, how="all")
   dict_donnees_insee[item[0]] = dataframe
   #print(item[0], str(dict_donnees_insee[item[0]].shape))
dataframes = dict_donnees_insee.values()
dataset_insee = pd.concat(dataframes, axis=0,keys=sheet_names).reset_index(level=1,drop=True)
dataset_insee.insert(0,"N°_Departement",dataset_insee["Zone_geographique"].apply(separer_numero_departement))
dataset_insee = dataset_insee.drop("Zone_geographique", axis=AXE_COLONNES)
display(dataset_insee)

## Traîtement des données manquantes sur l'emploi

In [ ]:
msno.bar(dataset_insee)
title = "Donnees_manquantes_emploi_insee_LA_AM_avant_traitement_donnees_AM"
plt.title(title)
plt.savefig(title + ".png")
plt.show()

In [ ]:
nan_values_dataframe = dataset_insee.isna()
missing_values_row_indexes = nan_values_dataframe[(nan_values_dataframe["2022T1"] == True)|(nan_values_dataframe["2022T2"] == True)|(nan_values_dataframe["2022T3"].isnull().any() == True)|(nan_values_dataframe["2022T4"] == True)].index
missing_values_row_indexes
rows_w_missing_values = dataset_insee.loc[missing_values_row_indexes]
interesting_columns = [column for column in rows_w_missing_values if "2022" in column]
for column in interesting_columns :
    print("\n"+ column)
    print(rows_w_missing_values[column].isna())

## Régression linéaire sur la série Tertiaire non marchand AM présentant des données manquantes
### Détermination de la droite de régression linéaire (d'ajustement)

In [ ]:
rows_w_missing_values.columns = range(rows_w_missing_values.columns.size)
regressor_wrappers_dict = {}
# Régression linéaire sur les données manquantes
previous_data_slice = slice(1,82)
target_slice = slice(82, None)
#Plaçage des features et des labels dans des tableaux à une dimension, Travail avec les données emploi Indus LA
x = rows_w_missing_values.columns[previous_data_slice].to_numpy().reshape(-1,1)
for i in range(len(rows_w_missing_values)):
    current_series_name = rows_w_missing_values.index[i]
    y = rows_w_missing_values.iloc[i,previous_data_slice].to_numpy().reshape(-1,1)
    #Découpage des données en jeux d'entraînement et de test
    #splitted_datasets_tuple = train_test_split(x, y, test_size=0.2, train_size=0.8)
    #Préparation des données à la régression linéaire
    #X_train, X_test, y_train, y_test = map(lambda dataset : dataset.reshape(1,dataset.shape[0]), splitted_datasets_tuple)
    
    regression_model = LinearRegression()
    # Adapter les données (entraînement du modèle)
    regression_model.fit(x,y)
    # Prédiction
    y_predicted = regression_model.predict(x)
    
    regressor_wrappers_dict[current_series_name] = RegressorWrapper(regression_model, x, y, y_predicted)
    r2 = r2_score(y, y_predicted)
    corr_coef = math.sqrt(r2)
    print(corr_coef)
    print(regression_model.intercept_)

In [ ]:
ticks_labels = list(dataset_insee.filter(axis=AXE_COLONNES, regex="\d{4}T4").columns)
x_y_axis_dataframe_slicing = slice(1,None)
title = "Séries avec des données manquantes"
for row_label in rows_w_missing_values.index:
    x = list(rows_w_missing_values.columns)[x_y_axis_dataframe_slicing]
    y = rows_w_missing_values.loc[row_label].iloc[x_y_axis_dataframe_slicing]
    plt.xticks(range(0,85,4),labels=ticks_labels)
    plt.plot(x,y,label=row_label)
for name,value in regressor_wrappers_dict.items():
    plt.plot(x[0:81], value.y_predicted, label="Régression linéaire " + name + " Équation : " + value.get_linear_equation())
plt.title(title)
plt.legend()
plt.savefig(title + ".png")
plt.show()

### Analyse des résidus pour l'emploi dans le secteur tertiaire en Alpes Maritimes

In [ ]:
tnm_reg_wrapper = regressor_wrappers_dict[rows_w_missing_values.index[1]]
residuals = tnm_reg_wrapper.get_residuals()
title = "Résidus Tertaire non marchand Alpes Maritimes (y - ymodèle)"
plt.xticks(range(0,85,4))
plt.title(title)
plt.plot(tnm_reg_wrapper.x.tolist(),residuals)
plt.savefig(title + ".png")

### Remplissage des valeurs manquantes avec les prédictions du modèle de Régression linéaire sur les données de l'emploi dans le secteur marchand en Alpes-Maritimes

In [ ]:
nan_values = rows_w_missing_values[rows_w_missing_values.columns[rows_w_missing_values.isnull().any()]]
for x in nan_values :
    row_label, column_label = ('Tertiaire non march AM OK',dataset_insee.columns[x])
    cell = dataset_insee.at[row_label, column_label]
    if cell != np.nan:
        dataset_insee.at[row_label, column_label] = tnm_reg_wrapper.predict(np.array([[x]]))

### Affichage de graphiques montrant les valeurs manquantes après le traitement des données du secteur Tertiaire non marchand Alpes Maritimes

In [ ]:
msno.bar(dataset_insee)
title = "Donnees_manquantes_emploi_insee_LA_AM_après_traitement_donnees_AM"
plt.title(title)
plt.savefig(title + ".png")
plt.show()

In [ ]:
nan_values_dataframe = dataset_insee.isna()
missing_values_row_indexes = nan_values_dataframe[(nan_values_dataframe["2022T1"] == True)|(nan_values_dataframe["2022T2"] == True)|(nan_values_dataframe["2022T3"].isnull().any() == True)|(nan_values_dataframe["2022T4"] == True)].index
missing_values_row_indexes
rows_w_missing_values = dataset_insee.loc[missing_values_row_indexes]
rows_w_missing_values.isna().sum(axis=AXE_COLONNES)

## Traitement des données manquantes sur la série Emploi Indus LA

### Essai de la méthode d'interpolation linéaire 

In [ ]:
dataset_insee = dataset_insee.interpolate(method="linear")
dataset_insee["2022T2"]

### Affichage des données manquantes après la méthode d'interpolation linéaire

In [ ]:
msno.bar(dataset_insee)
title = "Donnees_manquantes_emploi_insee_LA_AM_après_traitement_donnees_AM_+_Interpolation_linéaire"
plt.title(title)
plt.savefig(title + ".png")
plt.show()

## Affichage de graphiques décrivant les données

In [ ]:
dataset_insee_44 = dataset_insee[dataset_insee["N°_Departement"] == 44]
dataset_insee_44

In [ ]:
dataset_insee_06 = dataset_insee[dataset_insee["N°_Departement"] == 6]
dataset_insee_06

In [ ]:
chomage_LA = dataset_insee_44.loc['Chom LA'].iloc[1:]
chomage_LA

In [ ]:
dataframes = [dataset_insee_44, dataset_insee_06]
x_y_axis_dataframe_slicing = slice(1,None)
ticks_labels = list(dataset_insee.filter(axis=AXE_COLONNES, regex="\d{4}T4").columns)
for i in range(len(dataframes)):
    dataframe = dataframes[i]
    departement = "Loire_Atlantique" if i == 0 else "Alpes_Maritimes"
    title = "Emploi_" + departement
    for row_label in dataframe.index:
        x = list(dataframe.columns)[x_y_axis_dataframe_slicing]
        y = dataframe.loc[row_label].iloc[x_y_axis_dataframe_slicing]
        plt.xticks(range(0,85,4),labels=ticks_labels)
        plt.plot(x,y,label=row_label)
    plt.title(title)
    plt.legend()
    plt.savefig(title + ".png")
    plt.show()

In [ ]:
proportions = []
for i in range(len(dataframes)) :
    zone = "LA" if i == 0 else "AM"
    libelle_colonne_emploi = "Emploi {}".format(zone)
    libelle_colonne_num_departement = "N°_Departement"
    dataframe = dataframes[i].drop(libelle_colonne_num_departement, axis=AXE_COLONNES)
    display(afficher_noms_colonnes_avec_valeurs_manquantes(dataframe))
    libelles_lignes_emploi_et_chomage = [libelle_colonne_emploi,"Chom {}".format(zone)]
    serie_emploi_total = dataframe.loc[libelle_colonne_emploi]
    dataframe_sans_emploi_total_ni_chomage = dataframe.drop(libelles_lignes_emploi_et_chomage, axis=0)
    proportion_emploi_par_secteur = dataframe_sans_emploi_total_ni_chomage.divide(serie_emploi_total).mul(100)
    display(proportion_emploi_par_secteur)
    display(proportion_emploi_par_secteur.sum(axis=0))
    proportions.append(proportion_emploi_par_secteur)

In [ ]:
trimestres_interessants = ["2007T2","2012T2","2017T2","2022T2"]
dataset_insee_proportions_emploi_LA = proportions[0]
dataset_insee_proportions_emploi_AM = proportions[1]
proportions_donnees_elections_LA = dataset_insee_proportions_emploi_LA[trimestres_interessants]
proportions_donnees_elections_AM = dataset_insee_proportions_emploi_AM[trimestres_interessants]
display(proportions_donnees_elections_LA)
proportions_elections = [proportions_donnees_elections_LA,proportions_donnees_elections_AM]
display(proportions_elections)

In [ ]:
for i in range(len(proportions_elections)) :
    proportion = proportions_elections[i]
    zone = "Loire-Atlantique" if i == 0 else "Alpes-Maritimes"
    for column in proportion.columns:
        title = 'Répartition de l\'emploi par secteur en {} pendant le 1er tour des élections de {}'.format(zone, column[0:4])
        fig, ax = plt.subplots()
        plt.title(title)
        ax.pie(proportion[column], autopct='%1.1f%%')
        plt.savefig(title + ".png")

## Traitement des données des élections
### Obtention d'un jeu de données par candidats

In [ ]:
donnees_1er_tour_2007 = pd.read_excel("./data/Presidentielle_2007_Tour_1.xls", sheet_name="Départements T1")
donnees_1er_tour_2012 = pd.read_excel("./data/Presidentielle_2012_Tour_1.xls", sheet_name="Départements T1")
donnees_1er_tour_2017 = pd.read_excel("./data/Presidentielle_2017_Resultats_Tour_1_c.xls", sheet_name="Départements Tour 1")
donnees_1er_tour_2022 = pd.read_excel("./data/resultats-par-niveau-dpt-t1-france-entiere.xlsx")
global_columns = ['Code du département', 'Libellé du département', 'Inscrits',
       'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs et nuls',
       '% Blancs/Ins', '% Blancs/Vot', '% Nuls/Ins', '% Nuls/Vot', 'Exprimés',
       '% Exp/Ins', '% Exp/Vot']
donnees_1er_tour_2022 = donnees_1er_tour_2022.drop("Etat saisie", axis=AXE_COLONNES)
donnees_1er_tour_2022.insert(7,"Blancs et nuls",donnees_1er_tour_2022["Blancs"] + donnees_1er_tour_2022["Nuls"])

dataframes_2017_2022 = [donnees_1er_tour_2017, donnees_1er_tour_2022]
for i in range(len(dataframes_2017_2022)):
    dataframe = dataframes_2017_2022[i]
    display(dataframe.columns)
    percentage_of_null_votes_and_blank_ballots_among_registered_people_column_name = "% BlNuls/Ins"
    dataframe.insert(8,percentage_of_null_votes_and_blank_ballots_among_registered_people_column_name,(dataframe["Blancs et nuls"] / dataframe["Inscrits"]) * 100)    
    percentage_of_null_votes_and_blank_ballots_among_voters_column_name = "% BlNuls/Vot"
    dataframe.insert(9,percentage_of_null_votes_and_blank_ballots_among_voters_column_name,(dataframe["Blancs et nuls"] / dataframe["Votants"]) * 100)
    if i == 0 :
        global_columns.append(percentage_of_null_votes_and_blank_ballots_among_registered_people_column_name)
        global_columns.append(percentage_of_null_votes_and_blank_ballots_among_voters_column_name)
global_columns

In [ ]:

donnees_1er_tour_2017["Code du département"] = donnees_1er_tour_2017["Code du département"].astype(str)
donnees_1er_tour_2017["Code du département"].unique()
donnees_1er_tour_2017_44_06 = donnees_1er_tour_2017[(donnees_1er_tour_2017["Code du département"] == '44') | (donnees_1er_tour_2017["Code du département"] == '06')]
display(msno.bar(donnees_1er_tour_2017_44_06))
display(donnees_1er_tour_2017_44_06.columns)
#Isolement des candidats
candidates_df = []
columns = ['Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins', '% Voix/Exp']
for i in range(11):
    regex = '(Sexe|Nom|Prénom|Voix|% Voix/Ins|% Voix/Exp)'
    if i > 0 :
        regex = regex + '\.' + str(i)
        print(regex)
    regex = regex + '$'
    candidates = donnees_1er_tour_2017_44_06.filter(regex=regex)
    candidates.columns = columns
    display(candidates)
    candidates_df.append(candidates)
total_candidates = pd.concat(candidates_df, axis=0)
total_candidates
global_results = donnees_1er_tour_2017_44_06[global_columns]
display(global_results)

election_results_2017 = global_results.join(total_candidates, how='inner')
election_results_2017
#Assignation à une opinion politique
#Aggrégation des résultats

## Catégrorisation des candidats par bord politique

In [ ]:
donnes_securite_dept = pd.read_csv("./data/donnee-securité.csv",
                                   sep=';')
# Filtrer les données de sécurité sur lles départements 44 et 06
donnees_securite_44_06 = donnes_securite_dept[
 (donnes_securite_dept['Code.département'] == '6') | (donnes_securite_dept['Code.département'] == '06') | (
                donnes_securite_dept['Code.département'] == '44')]

display(donnees_securite_44_06)
display(donnees_securite_44_06)